In [ ]:
%cd ..

In [ ]:
from sklearn.metrics import cohen_kappa_score
import numpy as np
import pandas as pd
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from tqdm.auto import tqdm
from sklearn.metrics import recall_score, precision_score, f1_score
import nltk
from src.hatespeech.attack import load_attack
nltk.download('punkt')
pd.set_option('max_colwidth', None)
pd.set_option('max_rows', 200)

## Measure annotation agreement

In [ ]:
processed_dir = Path("data") / "processed"
final_dir = Path("data") / "final"
dan_path = processed_dir / "dr_offensive_annotated_dan.csv"
anders_path = processed_dir / "dr_offensive_annotated_anders.csv"

In [ ]:
dan_df = pd.read_csv(dan_path, sep="\t")
anders_df = pd.read_csv(anders_path, sep="\t")
dan_df.head()

In [ ]:
cohen_kappa_score(dan_df.label, anders_df.label)

## Visualise annotation agreement

In [ ]:
label_df = pd.concat([dan_df[["label"]], anders_df[["label"]]], axis=1)
label_df.columns = ["Dan", "Anders"]
label_df.head()

In [ ]:
def change_label_names(label: str) -> str:
    if "context" in label:
        return "Missing context"
    else:
        return label
label_df = label_df.applymap(change_label_names)
label_df.head()

In [ ]:
label_df.groupby(["Dan", "Anders"]).size().unstack(fill_value=0)

## Extract dataframe with agreed labels

In [ ]:
indices_with_agreement = label_df.query('Dan == Anders and Dan != "Missing context"').index.tolist()
agreement_df = dan_df.loc[indices_with_agreement][["text", "label"]].reset_index().rename(columns=dict(index="idx"))
agreement_df.head()

In [ ]:
agreement_df.label.value_counts()

In [ ]:
val_df_pos = agreement_df.query("label == 'Offensive'").sample(frac=0.5, random_state=4242)
val_df_neg = agreement_df.query("label == 'Not offensive'").sample(frac=0.5, random_state=4242)
val_df = pd.concat((val_df_pos, val_df_neg), axis=0).sample(frac=1.).reset_index(drop=True)
val_df.label.value_counts()

In [ ]:
test_df = agreement_df[~agreement_df.idx.isin(val_df.idx)]
test_df.label.value_counts()

In [ ]:
agreement_df.to_parquet(processed_dir / "dr_offensive_annotated_agreement.parquet")
val_df.to_parquet(final_dir / "dr_offensive_val.parquet")
test_df.to_parquet(final_dir / "dr_offensive_test.parquet")

In [ ]:
model_id = 'models/xlmr-base2'

# Load tokenizer and model
if model_id == 'attack':
    tok, model = load_attack()
else:
    tok = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSequenceClassification.from_pretrained(model_id)

# Get logits
logits = torch.stack(
    [get_logits(doc, tok, model) for doc in tqdm(val_df.text, leave=False)]
)

In [ ]:
# Add the logits and equivalent probabilities to the validation dataframe
val_df["model_logits"] = logits.tolist()
val_df["model_probs"] = torch.sigmoid(logits).tolist()
val_df.head()

In [ ]:
# Get the sample indices on which the model was wrong
wrong_idxs = (
    torch.nonzero((logits > 0) != torch.tensor(val_labels)).squeeze(1).tolist()
)

# Get the samples on which the model was wrong
wrong_df = val_df.loc[wrong_idxs]

# Sort the dataframe by absolute value of logits
wrong_df = wrong_df.sort_values(by='model_logits', key=lambda x: abs(x), ascending=False)

wrong_df